In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from datetime import datetime, timedelta

In [2]:
from faker import Faker
fake = Faker('en_IN')

In [3]:
import sqlalchemy, pymysql
from sqlalchemy.dialects import mysql
import pandas as pd
db = sqlalchemy.create_engine(
     sqlalchemy.engine.url.URL(
          drivername='mysql+pymysql',
          username='root',
          password="123456",
          host='34.75.22.238',
          database="sukyanya"
     )
)

C:\Users\sukanya.sahay\AppData\Local\Temp\ipykernel_2292\415947101.py:5: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  sqlalchemy.engine.url.URL(


In [4]:
qry1='select max(customerid) as maxcustid from CUSTOMER_MASTER;'
maxcustid= pd.read_sql(qry1,con=db)
maxcustid=maxcustid.iat[0,0]

In [5]:
qry2='select max(Productid) from PRODUCT_MASTER;'
maxprodid= pd.read_sql(qry2,con=db)
maxprodid=maxprodid.iat[0,0]

In [6]:
qry3='select max(orderid) from ORDER_DETAILS;'
orderid= pd.read_sql(qry3,con=db)
orderid=orderid.iat[0,0]
print(orderid)

20000


# incremental order details

In [7]:
order_detail_fields = ['orderid', 'customerid', 'order_status_update_timestamp', 'order_status']
orders_Received = pd.DataFrame(columns=order_detail_fields, index=range(1, 15001))

x=orderid+1
j=1
for i in range(1, 5001):
    f=fake.random_int(min=1, max=maxcustid)
    for k in range (j,j+3):
        orders_Received['orderid'][k] = x
        orders_Received['customerid'][k]=f

    orders_Received['order_status_update_timestamp'][j] = fake.date_time_this_year()
    orders_Received['order_status'][j] = 'Received'
    orders_Received['order_status_update_timestamp'][j+1] = orders_Received['order_status_update_timestamp'][j] + timedelta(seconds=fake.random_int(min=1, max=86400))
    orders_Received['order_status'][j+1] = 'In_progress'
    orders_Received['order_status_update_timestamp'][j+2] = orders_Received['order_status_update_timestamp'][j+1] + timedelta(seconds=fake.random_int(min=1, max=172800))
    orders_Received['order_status'][j+2] = 'Delivered'
    j+=3
    x+=1

In [8]:
orders_Received.head(-10)

,orderid,customerid,order_status_update_timestamp,order_status
1,20001,770,2022-11-04 05:19:54,Received
2,20001,770,2022-11-05 02:25:25,In_progress
3,20001,770,2022-11-05 21:16:05,Delivered
4,20002,292,2022-08-29 03:57:24,Received
5,20002,292,2022-08-29 13:05:08,In_progress
...,...,...,...,...
14986,24996,432,2022-05-02 17:51:22,Received
14987,24996,432,2022-05-03 11:42:10,In_progress
14988,24996,432,2022-05-04 11:21:10,Delivered
14989,24997,295,2022-05-10 23:32:07,Received


## loading

In [9]:
orders_Received.to_sql('ORDER_DETAILS', con=db, if_exists='append',index=False)

15000

# incremental order items

In [10]:
order_item_fields = ['orderid','productid','quantity']

In [11]:
order_items = pd.DataFrame(columns=order_item_fields, index = range(1,5001))

In [12]:
for i in range(1,5001):
    order_items['orderid'][i]=fake.random_element(orders_Received['orderid'])
    order_items['productid'][i]=fake.random_int(min=1, max=maxprodid)
    order_items['quantity'][i]=fake.random_int(min=1, max=20)

In [13]:
order_items = order_items.drop_duplicates(subset=['orderid', 'productid'],keep='first')

In [14]:
order_items.head(-10)

,orderid,productid,quantity
1,21009,86,15
2,24042,72,18
3,20185,49,13
4,22702,81,14
5,24677,96,6
...,...,...,...
4986,20725,73,8
4987,23024,50,12
4988,21971,50,13
4989,20672,3,15


## loading

In [15]:
order_items.to_sql('ORDER_ITEMS', con=db, if_exists='append',index=False)

4979